In [25]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [26]:
dropout = pd.read_csv("04 important_features.csv")
dropout.drop("Unnamed: 0", 1, inplace=True)
dropout.shape

(2750, 22)

In [27]:
dropout.head()

,grado_divertimento_allenamento,personalita_competitiva,agonismo,sport_passione,grado_conciliare_scuola_sport,sesso_maschio,calcio,sesso_femmina,personalita_tranquilla,divertimento,...,maschio_calcio,personalita_polemica,sport_agonistico_genitori,tipo_sport_significativo_squadra,frequenza_settimanale,tipo_sport_significativo_individuale,sport_bisogno_corpo_mente,personalita_aperta,femmina_pallavolo_nuoto,dropout
0,3,4,1,1,3,1,1,0,4,0,...,0,1,0,0,3,1,0,5,0,1
1,3,4,1,0,3,0,0,1,6,0,...,0,1,0,0,2,1,0,5,0,1
2,3,5,1,0,4,1,1,0,3,1,...,1,3,0,1,3,0,0,5,0,1
3,0,6,1,1,2,1,1,0,0,0,...,0,6,0,1,3,0,0,6,0,0
4,2,2,0,0,2,1,1,0,5,1,...,1,4,0,1,3,0,0,5,0,1


In [28]:
X = dropout.drop("dropout", 1)
y = dropout.dropout
models = dict()
X.shape, y.shape

((2750, 21), (2750,))

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

models['LogisticRegression FS'] = dict()
#params = {'C':[0.01, 0.1, 1, 10, 100], 'penalty':["l1", "l2"]}
params = {'C':[0.1], 'penalty':["l1"]}
clf = GridSearchCV(LogisticRegression(random_state=0), params, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                  refit="recall").fit(X, y)
models['LogisticRegression FS']['model'] = clf.best_estimator_
models['LogisticRegression FS']['params'] = clf.best_params_
models['LogisticRegression FS']['metrics'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="mean_test"), 2)
models['LogisticRegression FS']['cv_results'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="test").filter(like="split"), 2)
print(models['LogisticRegression FS']['params'])
print(models['LogisticRegression FS']['metrics'])

{'C': 0.1, 'penalty': 'l1'}
   mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1  \
0                0.69                 0.69              0.72          0.71   

   mean_test_roc_auc  
0               0.75  


In [30]:
mod = pd.DataFrame({'grado_divertimento_allenamento':X.grado_divertimento_allenamento, 
                   'personalita_competitiva':X.personalita_competitiva,
                   'agonismo':X.agonismo})
clf = LogisticRegression(random_state=0).fit(mod, y)
x1, x2, x3 = [], [], []
for gda in range(5):
    for pc in range(7):
        for ag in range(2):
            x1.append(gda)
            x2.append(pc)
            x3.append(ag)
df = pd.DataFrame({'grado_divertimento_allenamento':pd.Series(x1),
                 'personalita_competitiva':pd.Series(x2),
                 'agonismo':pd.Series(x3)})
df['prob_dropout'] = round(pd.DataFrame(clf.predict_proba(df)).iloc[:, 1], 2)
df.head()
#df.to_csv("LR model.csv")

,grado_divertimento_allenamento,personalita_competitiva,agonismo,prob_dropout
0,0,0,0,0.91
1,0,0,1,0.83
2,0,1,0,0.89
3,0,1,1,0.80
4,0,2,0,0.88


In [31]:
from sklearn.ensemble import RandomForestClassifier

models['RandomForest FS'] = dict()
params = {'criterion':['gini'], 'max_depth':[3], 'min_samples_leaf':[0.05], 'n_estimators':[500], 'bootstrap':[False]}
clf = GridSearchCV(RandomForestClassifier(random_state=0), params, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                  refit='recall').fit(X, y)
models['RandomForest FS']['model'] = clf.best_estimator_
models['RandomForest FS']['params'] = clf.best_params_
models['RandomForest FS']['metrics'] = round(pd.DataFrame(clf.cv_results_).query('rank_test_recall == 1').filter(like="mean_test"), 2)
models['RandomForest FS']['cv_results'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="test").filter(like="split"), 2)
print(models['RandomForest FS']['params'])
print(models['RandomForest FS']['metrics'])

{'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 0.05, 'n_estimators': 500}
   mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1  \
0                0.67                 0.66              0.71          0.69   

   mean_test_roc_auc  
0               0.75  


In [32]:
from sklearn.svm import SVC

models['SVM FS'] = dict()
#params = {'C':[0.1, 1, 10], 'kernel':["linear", "poly", "rbf"]}
params = {'C':[0.1], 'kernel':["poly"]}
clf = GridSearchCV(SVC(random_state=0, probability=True), params, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                  refit="recall").fit(X, y)
models['SVM FS']['model'] = clf.best_estimator_
models['SVM FS']['params'] = clf.best_params_
models['SVM FS']['metrics'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="mean_test"), 2)
models['SVM FS']['cv_results'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="test").filter(like="split"), 2)
print(models['SVM FS']['params'])
print(models['SVM FS']['metrics'])

{'C': 0.1, 'kernel': 'poly'}
   mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1  \
0                0.68                 0.66              0.79          0.72   

   mean_test_roc_auc  
0               0.75  


In [33]:
from sklearn.neighbors import KNeighborsClassifier

models['KNN FS'] = dict()
#params = {'n_neighbors':[50, 100, 200], 'p':[1, 2], 'weights':['uniform', 'distance']}
params = {'n_neighbors':[50], 'p':[2], 'weights':['distance']}
clf = GridSearchCV(KNeighborsClassifier(), params, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                  refit="recall").fit(X, y)
models['KNN FS']['model'] = clf.best_estimator_
models['KNN FS']['params'] = clf.best_params_
models['KNN FS']['metrics'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="mean_test"), 2)
models['KNN FS']['cv_results'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="test").filter(like="split"), 2)
print(models['KNN FS']['params'])
print(models['KNN FS']['metrics'])

{'n_neighbors': 50, 'p': 2, 'weights': 'distance'}
   mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1  \
0                0.68                 0.67              0.72           0.7   

   mean_test_roc_auc  
0               0.75  


In [34]:
from sklearn.neural_network import MLPClassifier

models['MLP FS'] = dict()
#params = {'hidden_layer_sizes':[(50, ), (100, ), (150, )], 'activation':['tanh', 'relu']}
params = {'hidden_layer_sizes':[(100, )], 'activation':['tanh']}
clf = GridSearchCV(MLPClassifier(random_state=0), params, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                  refit="recall").fit(X, y)
models['MLP FS']['model'] = clf.best_estimator_
models['MLP FS']['params'] = clf.best_params_
models['MLP FS']['metrics'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="mean_test"), 2)
models['MLP FS']['cv_results'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="test").filter(like="split"), 2)
print(models['MLP FS']['params'])
print(models['MLP FS']['metrics'])

{'activation': 'tanh', 'hidden_layer_sizes': (100,)}
   mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1  \
0                0.69                 0.67              0.77          0.72   

   mean_test_roc_auc  
0               0.76  


In [35]:
from sklearn.ensemble import VotingClassifier

models['VotingSoft FS'] = dict()
clf = GridSearchCV(VotingClassifier(estimators=[('lr', models['LogisticRegression FS']['model']),
                                                ('rf', models['RandomForest FS']['model']),
                                                ('svm', models['SVM FS']['model']),
                                                ('knn', models['KNN FS']['model']),
                                                ('mlp', models['MLP FS']['model'])], voting="soft"), {}, cv=10, 
                   scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], refit="recall").fit(X, y)
models['VotingSoft FS']['model'] = clf.best_estimator_
models['VotingSoft FS']['params'] = clf.best_params_
models['VotingSoft FS']['metrics'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="mean_test"), 2)
models['VotingSoft FS']['cv_results'] = round(pd.DataFrame(clf.cv_results_).query("rank_test_recall == 1").filter(like="test").filter(like="split"), 2)
print(models['VotingSoft FS']['metrics'])

   mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1  \
0                 0.7                 0.68              0.76          0.72   

   mean_test_roc_auc  
0               0.77  


In [37]:
import pickle
pickle.dump(models['LogisticRegression FS'], open('LogisticRegression FS', 'wb'))
pickle.dump(models['RandomForest FS'], open('RandomForest FS', 'wb'))
pickle.dump(models['SVM FS'], open('SVM FS', 'wb'))
pickle.dump(models['KNN FS'], open('KNN FS', 'wb'))
pickle.dump(models['MLP FS'], open('MLP FS', 'wb'))
pickle.dump(models['VotingSoft FS'], open('VotingSoft FS', 'wb'))